In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pathlib
import pickle
import random

sys.path.append('../..')

In [3]:
import pandas as pd

In [4]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp, get_ms1_peaks
from pals.feature_extraction import DataSource
from pals.PLAGE import PLAGE
from pals.ORA import ORA
from pals.common import *
from pals.RunMummichog import *

2021-09-27 11:58:42.788 | WARNING  | pals.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2021-09-27 11:58:42.791 | WARNING  | pals.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2021-09-27 11:58:42.800 | INFO     | pals.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Beer Analysis

### Load data

Generate token by logging in to PiMP

In [5]:
# username = 'joewandy' # PiMP username
# password = 'enter' # PiMP password
# host = 'localhost:8000' # server address and port
# token = get_authentication_token(host, username, password)

Assume token is stored in environmental variable *PIMP_API_TOKEN*

In [5]:
#token goes here
PIMP_API_TOKEN = token

In [6]:
analysis_id = 14 # CSF_FIND

Get peak m/z and RT for mummichog

In [7]:
ms1_df = get_ms1_peaks(token, PIMP_HOST, analysis_id)
ms1_df

2021-09-27 11:59:01.922 | DEBUG    | pals.pimp_tools:get_data:33 - http://polyomics.mvls.gla.ac.uk/export/get_ms1_peaks?analysis_id=14


,pid,sec_id,mass,rt,polarity,cmpd_id,formula,adduct,identified,rc_id,compound,db,identifier,frank_annot,inchikey
0,36808,623,97.991359,1394.926682,positive,1,CaO,M+ACN+H,False,176516,Calcium oxide,kegg,C13140,NaN,BRPQOXSCLDDYGP-UHFFFAOYSA-N
1,36723,538,97.991360,1354.241768,positive,1,CaO,M+ACN+H,False,176362,Calcium oxide,kegg,C13140,NaN,BRPQOXSCLDDYGP-UHFFFAOYSA-N
2,36739,554,97.991357,1219.664623,positive,1,CaO,M+ACN+H,False,176374,Calcium oxide,kegg,C13140,NaN,BRPQOXSCLDDYGP-UHFFFAOYSA-N
3,37132,947,97.991358,1174.563337,positive,1,CaO,M+ACN+H,False,177868,Calcium oxide,kegg,C13140,NaN,BRPQOXSCLDDYGP-UHFFFAOYSA-N
4,37309,1053,171.005747,1049.387860,positive,2,C5H8O4,M+K,False,178414,Monoethyl malonic acid,hmdb,HMDB00576,NaN,HGINADPHJQTSKN-UHFFFAOYSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6409,37457,1126,168.065645,731.447731,positive,7648,C8H9NO3,M+H,False,179319,Pyridoxal,kegg,C00250,NaN,None
6410,37513,1148,114.055046,437.709974,positive,7649,C5H7NO2,M+H,False,179482,delta-1-Pyrroline-5-carboxylate,kegg,C04322,NaN,None
6411,37788,1278,332.921902,909.031581,negative,7651,C10H14BrCl3O,M-H,False,179895,Costatol,kegg,C17103,NaN,None
6412,38501,1611,332.921899,981.516007,negative,7654,C10H14BrCl3O,M-H,False,181714,Costatol,kegg,C17103,NaN,None


In [8]:
ms1_df.rename(columns={
    'pid': 'row_id',
    'mass': 'm/z',
    'rt': 'retention_time'
}, inplace=True)
ms1_df[['row_id', 'm/z', 'retention_time']].set_index('row_id')

,m/z,retention_time
row_id,,
36808,97.991359,1394.926682
36723,97.991360,1354.241768
36739,97.991357,1219.664623
37132,97.991358,1174.563337
37309,171.005747,1049.387860
...,...,...
37457,168.065645,731.447731
37513,114.055046,437.709974
37788,332.921902,909.031581


#Get the rest of the dataframes we need to construct a DataSource object

In [9]:
int_df, annotation_df, experimental_design = download_from_pimp(token, PIMP_HOST, analysis_id, 'kegg')

2021-09-27 11:59:17.017 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: C:\Users\simon\AppData\Local\Temp\pimp_analysis_14.p


In [10]:
int_df
int_df.to_csv('intensity_csv_real.csv')

In [11]:
annotation_df

,entity_id
row_id,
37309,C00111
36397,C00111
37231,C00111
36199,C00111
36883,C00111
...,...
37457,C00250
37513,C04322
37788,C17103


In [12]:
experimental_design

{'comparisons': [{'case': 'Stage_1',
   'control': 'Stage_2',
   'name': 'Stage_1/Stage_2'},
  {'case': 'Stage_2', 'control': 'Control', 'name': 'Stage_2/Control'},
  {'case': 'Stage_1', 'control': 'Control', 'name': 'Stage_1/Control'}],
 'groups': {'Control': ['A020C.mzXML',
   'A014C.mzXML',
   'A009C.mzXML',
   'A011C.mzXML',
   'A003C.mzXML',
   'A015C.mzXML',
   'A016C.mzXML',
   'A004C.mzXML',
   'A001C.mzXML',
   'B001C.mzXML',
   'A008C.mzXML',
   'A010C.mzXML',
   'B002C.mzXML',
   'A005C.mzXML',
   'A019C.mzXML',
   'A017C.mzXML',
   'A013C.mzXML'],
  'Stage_1': ['A019P.mzXML',
   'A053P.mzXML',
   'A010P.mzXML',
   'A011P.mzXML',
   'A039P.mzXML',
   'A107P.mzXML',
   'B016P.mzXML',
   'A021P.mzXML',
   'A013P.mzXML',
   'A057P.mzXML',
   'A024P.mzXML',
   'A068P.mzXML',
   'A060P.mzXML',
   'A012P.mzXML',
   'A008P.mzXML',
   'A058P.mzXML',
   'A062P.mzXML',
   'A061P.mzXML',
   'B012P.mzXML',
   'B014P.mzXML'],
  'Stage_2': ['A023P.mzXML',
   'A044P.mzXML',
   'A027P.mzXML

In [13]:
int_df = int_df.reindex(sorted(int_df.columns), axis=1)
int_df

,A001C.mzXML,A001P.mzXML,A002P.mzXML,A003C.mzXML,A004C.mzXML,A005C.mzXML,A008C.mzXML,A008P.mzXML,A009C.mzXML,A010C.mzXML,...,A081P.mzXML,A082P.mzXML,A083P.mzXML,A107P.mzXML,A118P.mzXML,B001C.mzXML,B002C.mzXML,B012P.mzXML,B014P.mzXML,B016P.mzXML
row_id,,,,,,,,,,,,,,,,,,,,,
36186,6.127151e+08,4.087233e+08,3.565927e+08,5.758748e+08,6.277334e+08,3.997072e+08,6.214990e+08,5.455244e+08,5.228084e+08,5.573633e+08,...,3.491120e+08,4.124747e+08,5.055433e+08,6.136892e+08,4.967120e+08,5.840201e+08,4.795199e+08,5.415740e+08,4.771584e+08,4.667319e+08
36187,2.726796e+08,2.220560e+08,1.879616e+08,2.548970e+08,2.747774e+08,2.045973e+08,2.711476e+08,2.442694e+08,2.463833e+08,2.625147e+08,...,1.867164e+08,2.098798e+08,2.447424e+08,2.905365e+08,2.302146e+08,2.659387e+08,2.315487e+08,2.505285e+08,2.349865e+08,2.317964e+08
36188,1.420786e+08,9.454549e+07,9.779530e+07,1.289906e+08,1.461389e+08,9.901544e+07,1.371838e+08,1.314715e+08,1.251233e+08,1.350838e+08,...,8.872685e+07,1.068299e+08,1.270307e+08,1.452775e+08,1.201612e+08,1.376156e+08,1.226677e+08,1.267788e+08,1.167567e+08,1.200663e+08
36189,1.259999e+08,9.654570e+07,8.381609e+07,1.304112e+08,1.298402e+08,9.358969e+07,1.361706e+08,1.323530e+08,1.184184e+08,1.265139e+08,...,8.037537e+07,9.417155e+07,1.202441e+08,1.345063e+08,1.137046e+08,1.217804e+08,1.144381e+08,1.171860e+08,1.171399e+08,1.078630e+08
36190,2.016064e+07,1.617546e+07,1.397474e+07,1.672281e+07,1.887251e+07,1.451794e+07,1.786245e+07,1.676201e+07,1.718698e+07,1.831481e+07,...,1.448752e+07,1.448341e+07,1.663064e+07,2.219927e+07,1.614626e+07,1.827413e+07,1.688614e+07,1.891526e+07,1.695035e+07,1.583075e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39727,6.017799e+04,3.305169e+04,3.774021e+04,4.254625e+04,5.122880e+04,5.144634e+04,6.651510e+04,3.383649e+04,5.543958e+04,2.775504e+04,...,4.452187e+04,4.139891e+04,3.937648e+04,5.478839e+04,4.103015e+04,4.886858e+04,5.120511e+04,2.760448e+04,4.022374e+04,3.861848e+04
39728,5.524893e+04,4.471079e+04,4.417618e+04,4.142946e+04,4.632614e+04,3.615989e+04,4.392817e+04,4.920196e+04,4.222943e+04,4.898637e+04,...,4.672691e+04,3.589568e+04,3.784676e+04,5.113102e+04,4.134180e+04,5.073580e+04,5.193784e+04,5.656038e+04,3.906630e+04,5.422961e+04
39729,3.087718e+04,3.764938e+04,4.133501e+04,5.282910e+04,3.678989e+04,3.919970e+04,5.085458e+04,5.502905e+04,4.600038e+04,5.198357e+04,...,5.949855e+04,3.238906e+04,3.844145e+04,4.886430e+04,4.276266e+04,5.208983e+04,5.099948e+04,5.142990e+04,5.339018e+04,5.539751e+04


In [14]:
lookup = {}
for group in experimental_design['groups']:
    samples = experimental_design['groups'][group]
    for sample in samples:
        lookup[sample] = group

for sample in int_df.columns:
#     print('%s,%s' % (sample, lookup[sample]))
    print('%s' % (lookup[sample]))

Control
Stage_2
Stage_2
Control
Control
Control
Control
Stage_1
Control
Control
Stage_1
Control
Stage_1
Stage_1
Control
Stage_1
Control
Stage_2
Control
Control
Stage_2
Control
Stage_2
Stage_2
Control
Stage_1
Control
Stage_1
Stage_2
Stage_1
Stage_2
Stage_1
Stage_2
Stage_2
Stage_2
Stage_1
Stage_2
Stage_1
Stage_1
Stage_1
Stage_1
Stage_1
Stage_2
Stage_2
Stage_1
Stage_2
Stage_2
Stage_2
Stage_2
Stage_2
Stage_1
Stage_2
Control
Control
Stage_1
Stage_1
Stage_1


In [15]:
int_df.to_csv('int_df.csv')
annotation_df.to_csv('annotation_df.csv')

### PALS analysis using KEGG database exported from PiMP

In [40]:
comparisons = [{'case' : 'Stage_1', 'control' : 'Stage_2', 'name' : 'Stage_1/Stage_2'}]

In [41]:
#unchanged analysis

unchanged_int = pd.read_csv('int_df.csv')

ds = DataSource(unchanged_int, annotation_df, experimental_design, DATABASE_PIMP_KEGG, mz_rt_df=ms1_df, comparisons=comparisons)

2021-09-27 12:10:58.766 | DEBUG    | pals.feature_extraction:__init__:52 - Using PiMP_KEGG as database
2021-09-27 12:10:58.767 | DEBUG    | pals.loader:load_data:42 - Loading C:\Users\simon\PALS\pals\data\PiMP_KEGG.json.zip
2021-09-27 12:10:58.804 | DEBUG    | pals.feature_extraction:__init__:80 - Mapping pathway to unique ids
2021-09-27 12:10:58.819 | DEBUG    | pals.feature_extraction:__init__:94 - Creating dataset to pathway mapping
2021-09-27 12:10:58.964 | DEBUG    | pals.feature_extraction:__init__:122 - Computing unique id counts


In [42]:
mummichog_ds = ds.create_mummichog_ds()

mummichog_pa = MummichogPathwayAnalysis(mummichog_ds)
results_obj = mummichog_pa.get_results()

Read 2316 features as reference list.
Using 816 features (p < 0.100000) as significant list.
Read 2309 features as reference list.
Using 815 features (p < 0.100000) as significant list.


Got 159 cpd2mzFeatures
Got 202 ListOfEmpiricalCompounds
Got 91 merged ListOfEmpiricalCompounds
Got 40 final ListOfEmpiricalCompounds



Pathway Analysis...


Got 159 cpd2mzFeatures
Got 202 ListOfEmpiricalCompounds
Got 91 merged ListOfEmpiricalCompounds
Got 40 final ListOfEmpiricalCompounds



Pathway Analysis...
Query number of significant compounds = 38 compounds
Resampling, 100 permutations to estimate background ...


 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100


Pathway background is estimated on 11900 random pathway values
Query number of significant compounds = 34 compounds
Resampling, 100 permutations to estimate background ...


 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100


Pathway background is estimated on 11900 random pathway values


In [43]:
pathway_df = results_obj[0].construct_pathway_enrich_df()
pathway_df

0                                            pathway overlap_size  \
1  Glycine, serine, alanine and threonine metabolism            6   
2                                Tyrosine metabolism            6   
3                     Glycerophospholipid metabolism            5   
4                              Pyrimidine metabolism            5   
5                              Tryptophan metabolism            4   

0 pathway_size               p-value       overlap_Empirical  \
1            6   0.02386354087891774  E49,E8,E41,E50,E13,E51   
2            6   0.02386354087891774  E87,E8,E11,E91,E84,E86   
3            5  0.049743719015208805        E6,E79,E13,E5,E4   
4            5  0.049743719015208805      E25,E8,E11,E34,E24   
5            4   0.06495252499789933           E83,E7,E8,E32   

0                              overlap_features (id)  \
1  C00719/C00431/C00183,C00213/C00041/C00099/C001...   
2  C02505/C02558,C00213/C00041/C00099/C00133,C000...   
3  C00661/C00111/C00118,C01595,C

,pathway,overlap_size,pathway_size,p-value,overlap_Empirical,overlap_features (id),overlap_features (name)
1,"Glycine, serine, alanine and threonine metabolism",6,6,0.02386354087891774,"E49,E8,E41,E50,E13,E51","C00719/C00431/C00183,C00213/C00041/C00099/C001...",Glycine betaine/5-Aminopentanoate; 5-Aminopent...
2,Tyrosine metabolism,6,6,0.02386354087891774,"E87,E8,E11,E91,E84,E86","C02505/C02558,C00213/C00041/C00099/C00133,C000...",2-Phenylacetamide/N-Acetylarylamine$Sarcosine;...
3,Glycerophospholipid metabolism,5,5,0.049743719015208805,"E6,E79,E13,E5,E4","C00661/C00111/C00118,C01595,C00065/C00740,C006...",DL-Glyceraldehyde 3-phosphate/Dihydroxyacetone...
4,Pyrimidine metabolism,5,5,0.049743719015208805,"E25,E8,E11,E34,E24","C00906,C00213/C00041/C00099/C00133,C00064/C051...","5,6-Dihydrothymine$Sarcosine; N-Methylglycine/..."
5,Tryptophan metabolism,4,4,0.06495252499789933,"E83,E7,E8,E32",",,C00213/C00041/C00099/C00133,",$$Sarcosine; N-Methylglycine/L-Alanine/beta-Al...
...,...,...,...,...,...,...,...
115,C21-steroid hormone biosynthesis and metabolism,0,0,0.4083690446180993,,,
116,Starch and Sucrose Metabolism,0,0,0.4083690446180993,,,
117,Trihydroxycoprostanoyl-CoA beta-oxidation,0,0,0.4083690446180993,,,
118,Omega-6 fatty acid metabolism,0,0,0.4083690446180993,,,


In [187]:
pathway_df.to_csv('baseline.csv')

In [296]:
#make a copy df
int_df = unchanged_int
decoy_df = ms1_df.copy()

In [297]:
#make a list of random indexes between 0-2378
rand_idx = random.sample(range(0,2378), 10)

print(len(rand_idx))

#then we generate mz and rt data and 1 peak intensity result per index item


10


In [298]:
#we need to look up the intensity df and make a random intensity value and make it the same for every condition
min_intensity = int_df.min()[0]
max_intensity = int_df.max()[0]


int_df

,row_id,A001C.mzXML,A001P.mzXML,A002P.mzXML,A003C.mzXML,A004C.mzXML,A005C.mzXML,A008C.mzXML,A008P.mzXML,A009C.mzXML,...,A081P.mzXML,A082P.mzXML,A083P.mzXML,A107P.mzXML,A118P.mzXML,B001C.mzXML,B002C.mzXML,B012P.mzXML,B014P.mzXML,B016P.mzXML
0,36186,6.127151e+08,4.087233e+08,3.565927e+08,5.758748e+08,6.277334e+08,3.997072e+08,6.214990e+08,5.455244e+08,5.228084e+08,...,3.491120e+08,4.124747e+08,5.055433e+08,6.136892e+08,4.967120e+08,5.840201e+08,4.795199e+08,5.415740e+08,4.771584e+08,4.667319e+08
1,36187,2.726796e+08,2.220560e+08,1.879616e+08,2.548970e+08,2.747774e+08,2.045973e+08,2.711476e+08,2.442694e+08,2.463833e+08,...,1.867164e+08,2.098798e+08,2.447424e+08,2.905365e+08,2.302146e+08,2.659387e+08,2.315487e+08,2.505285e+08,2.349865e+08,2.317964e+08
2,36188,1.420786e+08,9.454549e+07,9.779530e+07,1.289906e+08,1.461389e+08,9.901544e+07,1.371838e+08,1.314715e+08,1.251233e+08,...,8.872685e+07,1.068299e+08,1.270307e+08,1.452775e+08,1.201612e+08,1.376156e+08,1.226677e+08,1.267788e+08,1.167567e+08,1.200663e+08
3,36189,1.259999e+08,9.654570e+07,8.381609e+07,1.304112e+08,1.298402e+08,9.358969e+07,1.361706e+08,1.323530e+08,1.184184e+08,...,8.037537e+07,9.417155e+07,1.202441e+08,1.345063e+08,1.137046e+08,1.217804e+08,1.144381e+08,1.171860e+08,1.171399e+08,1.078630e+08
4,36190,2.016064e+07,1.617546e+07,1.397474e+07,1.672281e+07,1.887251e+07,1.451794e+07,1.786245e+07,1.676201e+07,1.718698e+07,...,1.448752e+07,1.448341e+07,1.663064e+07,2.219927e+07,1.614626e+07,1.827413e+07,1.688614e+07,1.891526e+07,1.695035e+07,1.583075e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,39727,6.017799e+04,3.305169e+04,3.774021e+04,4.254625e+04,5.122880e+04,5.144634e+04,6.651510e+04,3.383649e+04,5.543958e+04,...,4.452187e+04,4.139891e+04,3.937648e+04,5.478839e+04,4.103015e+04,4.886858e+04,5.120511e+04,2.760448e+04,4.022374e+04,3.861848e+04
2375,39728,5.524893e+04,4.471079e+04,4.417618e+04,4.142946e+04,4.632614e+04,3.615989e+04,4.392817e+04,4.920196e+04,4.222943e+04,...,4.672691e+04,3.589568e+04,3.784676e+04,5.113102e+04,4.134180e+04,5.073580e+04,5.193784e+04,5.656038e+04,3.906630e+04,5.422961e+04
2376,39729,3.087718e+04,3.764938e+04,4.133501e+04,5.282910e+04,3.678989e+04,3.919970e+04,5.085458e+04,5.502905e+04,4.600038e+04,...,5.949855e+04,3.238906e+04,3.844145e+04,4.886430e+04,4.276266e+04,5.208983e+04,5.099948e+04,5.142990e+04,5.339018e+04,5.539751e+04
2377,39730,3.555681e+04,3.901761e+04,4.225745e+04,5.355777e+04,3.926422e+04,4.596566e+04,4.080913e+04,4.508458e+04,4.201191e+04,...,4.450496e+04,4.379064e+04,4.802203e+04,2.490964e+04,4.614917e+04,3.673308e+04,4.876858e+04,4.173100e+04,3.298996e+04,3.775479e+04


In [299]:
for index in rand_idx:
    int_df.iloc[index] = np.random.randint(min_intensity, max_intensity)
    

print(rand_idx)
print(int_df.iloc[5])    

[1856, 180, 79, 588, 1832, 151, 1512, 1612, 1026, 1795]
row_id            36191.0
A001C.mzXML    17529516.0
A001P.mzXML    12662189.0
A002P.mzXML    11387804.0
A003C.mzXML    16389408.0
A004C.mzXML    19201358.0
A005C.mzXML    13794260.0
A008C.mzXML    18322500.0
A008P.mzXML    16015159.0
A009C.mzXML    14964422.0
A010C.mzXML    17183404.0
A010P.mzXML    16348388.0
A011C.mzXML    16053271.0
A011P.mzXML    16096133.0
A012P.mzXML    16057304.0
A013C.mzXML    15354356.0
A013P.mzXML    15318960.0
A014C.mzXML    16610922.0
A014P.mzXML    16896688.0
A015C.mzXML    12697054.0
A016C.mzXML    18576956.0
A016P.mzXML    16132879.0
A017C.mzXML    14887124.0
A017P.mzXML    15916723.0
A018P.mzXML    16260772.0
A019C.mzXML    16031827.0
A019P.mzXML    14764201.0
A020C.mzXML    14116381.0
A021P.mzXML    15402255.0
A023P.mzXML    14418096.0
A024P.mzXML    18234462.0
A027P.mzXML    14898689.0
A039P.mzXML    15818956.0
A044P.mzXML    17970250.0
A046P.mzXML    16582540.0
A048P.mzXML    16334683.0
A053P.mz

In [290]:
int_df

,row_id,A001C.mzXML,A001P.mzXML,A002P.mzXML,A003C.mzXML,A004C.mzXML,A005C.mzXML,A008C.mzXML,A008P.mzXML,A009C.mzXML,...,A081P.mzXML,A082P.mzXML,A083P.mzXML,A107P.mzXML,A118P.mzXML,B001C.mzXML,B002C.mzXML,B012P.mzXML,B014P.mzXML,B016P.mzXML
0,36186,6.127151e+08,4.087233e+08,3.565927e+08,5.758748e+08,6.277334e+08,3.997072e+08,6.214990e+08,5.455244e+08,5.228084e+08,...,3.491120e+08,4.124747e+08,5.055433e+08,6.136892e+08,4.967120e+08,5.840201e+08,4.795199e+08,5.415740e+08,4.771584e+08,4.667319e+08
1,36187,2.726796e+08,2.220560e+08,1.879616e+08,2.548970e+08,2.747774e+08,2.045973e+08,2.711476e+08,2.442694e+08,2.463833e+08,...,1.867164e+08,2.098798e+08,2.447424e+08,2.905365e+08,2.302146e+08,2.659387e+08,2.315487e+08,2.505285e+08,2.349865e+08,2.317964e+08
2,39044,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,...,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04,3.904400e+04
3,36189,1.259999e+08,9.654570e+07,8.381609e+07,1.304112e+08,1.298402e+08,9.358969e+07,1.361706e+08,1.323530e+08,1.184184e+08,...,8.037537e+07,9.417155e+07,1.202441e+08,1.345063e+08,1.137046e+08,1.217804e+08,1.144381e+08,1.171860e+08,1.171399e+08,1.078630e+08
4,36190,2.016064e+07,1.617546e+07,1.397474e+07,1.672281e+07,1.887251e+07,1.451794e+07,1.786245e+07,1.676201e+07,1.718698e+07,...,1.448752e+07,1.448341e+07,1.663064e+07,2.219927e+07,1.614626e+07,1.827413e+07,1.688614e+07,1.891526e+07,1.695035e+07,1.583075e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,39101,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,...,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04,3.910100e+04
2375,39728,5.524893e+04,4.471079e+04,4.417618e+04,4.142946e+04,4.632614e+04,3.615989e+04,4.392817e+04,4.920196e+04,4.222943e+04,...,4.672691e+04,3.589568e+04,3.784676e+04,5.113102e+04,4.134180e+04,5.073580e+04,5.193784e+04,5.656038e+04,3.906630e+04,5.422961e+04
2376,39729,3.087718e+04,3.764938e+04,4.133501e+04,5.282910e+04,3.678989e+04,3.919970e+04,5.085458e+04,5.502905e+04,4.600038e+04,...,5.949855e+04,3.238906e+04,3.844145e+04,4.886430e+04,4.276266e+04,5.208983e+04,5.099948e+04,5.142990e+04,5.339018e+04,5.539751e+04
2377,39730,3.555681e+04,3.901761e+04,4.225745e+04,5.355777e+04,3.926422e+04,4.596566e+04,4.080913e+04,4.508458e+04,4.201191e+04,...,4.450496e+04,4.379064e+04,4.802203e+04,2.490964e+04,4.614917e+04,3.673308e+04,4.876858e+04,4.173100e+04,3.298996e+04,3.775479e+04


In [291]:
#reconstruct mummichog datasource

decoy_mzdf = decoy_df[['row_id', 'm/z', 'retention_time']].copy()

decoy_mzdf[['row_id', 'm/z', 'retention_time']].set_index('row_id')




,m/z,retention_time
row_id,,
36739,97.991357,1219.664623
37132,97.991358,1174.563337
36808,97.991359,1394.926682
36723,97.991360,1354.241768
37231,171.005734,1148.437071
...,...,...
37457,168.065645,731.447731
37513,114.055046,437.709974
37788,332.921902,909.031581


In [313]:
decoy_data = DataSource(int_df, annotation_df, experimental_design, DATABASE_PIMP_KEGG, mz_rt_df=decoy_mzdf, comparisons=comparisons)
decoy_ds = decoy_data.create_mummichog_ds()

2021-09-24 23:20:28.285 | DEBUG    | pals.feature_extraction:__init__:52 - Using PiMP_KEGG as database
2021-09-24 23:20:28.286 | DEBUG    | pals.loader:load_data:42 - Loading C:\Users\Simon\PALS\pals\data\PiMP_KEGG.json.zip
2021-09-24 23:20:28.320 | DEBUG    | pals.feature_extraction:__init__:80 - Mapping pathway to unique ids
2021-09-24 23:20:28.331 | DEBUG    | pals.feature_extraction:__init__:94 - Creating dataset to pathway mapping
2021-09-24 23:20:28.445 | DEBUG    | pals.feature_extraction:__init__:122 - Computing unique id counts
Read 2315 features as reference list.
Using 611 features (p < 0.050000) as significant list.
Read 2308 features as reference list.
Using 604 features (p < 0.050000) as significant list.


In [263]:
decoy_pa = MummichogPathwayAnalysis(decoy_ds)
results_obj = decoy_pa.get_results()

a = results_obj[0].construct_pathway_enrich_df()



Pathway Analysis...

Pathway Analysis...
Query number of significant compounds = 37 compounds
Resampling, 100 permutations to estimate background ...


Got 159 cpd2mzFeatures
Got 202 ListOfEmpiricalCompounds
Got 91 merged ListOfEmpiricalCompounds
Got 40 final ListOfEmpiricalCompounds
Got 159 cpd2mzFeatures
Got 202 ListOfEmpiricalCompounds
Got 91 merged ListOfEmpiricalCompounds
Got 40 final ListOfEmpiricalCompounds
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100


Pathway background is estimated on 11900 random pathway values
Query number of significant compounds = 29 compounds
Resampling, 100 permutations to estimate background ...


 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100


Pathway background is estimated on 11900 random pathway values


In [264]:
a

,pathway,overlap_size,pathway_size,p-value,overlap_Empirical,overlap_features (id),overlap_features (name)
1,"Glycine, serine, alanine and threonine metabolism",6,6,0.006722124191244433,"E41,E49,E14,E8,E50,E51",",,,,,",$$$$$
2,Tyrosine metabolism,6,6,0.006722124191244433,"E84,E87,E86,E8,E11,E91",",,,,,",$$$$$
3,Glycerophospholipid metabolism,5,5,0.016553230820939416,"E79,E14,E4,E5,E6",",,,,",$$$$
4,Tryptophan metabolism,4,4,0.025964204688681623,"E32,E8,E83,E7",",,,",$$$
5,"Valine, leucine and isoleucine degradation",4,4,0.025964204688681623,"E49,E50,E82,E51",",,,",$$$
...,...,...,...,...,...,...,...
115,C21-steroid hormone biosynthesis and metabolism,0,0,0.3887908579110999,,,
116,Starch and Sucrose Metabolism,0,0,0.3887908579110999,,,
117,Trihydroxycoprostanoyl-CoA beta-oxidation,0,0,0.3887908579110999,,,
118,Omega-6 fatty acid metabolism,0,0,0.3887908579110999,,,


In [265]:
a.to_csv('decoy500.csv')

In [31]:
a.to_csv('decoy_nomz.csv')

In [314]:
ora = PLAGE(decoy_data)

new = ora.get_results()

new

2021-09-24 23:20:40.946 | DEBUG    | pals.PLAGE:__init__:31 - PLAGE initialised
2021-09-24 23:20:41.084 | DEBUG    | pals.preprocessing:process:17 - Replacing negative and zero values with NaN
2021-09-24 23:20:41.096 | DEBUG    | pals.preprocessing:process:29 - Performing min-value imputation
2021-09-24 23:20:41.109 | DEBUG    | pals.preprocessing:process:42 - Performing row average imputation
2021-09-24 23:20:41.131 | DEBUG    | pals.preprocessing:process:70 - Applying log normalisation
2021-09-24 23:20:41.135 | DEBUG    | pals.preprocessing:process:77 - Scaling the data across the sample: zero mean and unit variance
2021-09-24 23:20:41.187 | DEBUG    | pals.PLAGE:get_plage_activity_df:87 - Mean values of the rows in the DF is [ 0. -0. -0. ...  0.  0. -0.]
2021-09-24 23:20:41.188 | DEBUG    | pals.PLAGE:get_plage_activity_df:88 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]


KeyError: "None of [Int64Index([37468, 36428, 36798, 37122, 36377, 37137, 39334, 37117, 37508,\n            39296, 39296],\n           dtype='int64')] are in the [index]"